In [1]:
data = r"C:\Users\muham\OneDrive - Universitas Airlangga\Semester 6\Sistem Temu Kembali Informasi\Tugas dan Latihan\Final tugas akhir\ALL FILE CSV\all_file.csv"
stopword_id = r'C:\Users\muham\OneDrive - Universitas Airlangga\Semester 6\Sistem Temu Kembali Informasi\Tugas dan Latihan\Final tugas akhir\IR\stopwords-id.txt'

In [4]:
import pandas as pd
import numpy as np
import string
import re
import shutil
import os
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle

# Membaca file CSV
df = pd.read_csv(data)

# Inisialisasi TfidfVectorizer dengan stop words bahasa Indonesia
with open(stopword_id, 'r') as f:
    stop_words_id = f.read().splitlines()

tfidf = TfidfVectorizer(stop_words=stop_words_id, max_df=0.85, min_df=2, ngram_range=(1, 2))

# Melakukan fit dan transformasi pada kolom Teks
tfidf_matrix = tfidf.fit_transform(df['Teks'])

def preprocess_text(text, stemmer):
    # Pastikan text adalah string
    if not isinstance(text, str):
        text = str(text)
    
    # Menghilangkan karakter berulang
    text = re.sub(r'(.)\\1+', r'\\1', text)
    
    # Menghilangkan angka
    text = ''.join([i for i in text if not i.isdigit()])
    
    # Menghilangkan tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Mengubah teks menjadi huruf kecil
    text = text.lower()
    
    # Melakukan stemming pada teks
    text = stemmer.stem(text)
    
    return text

def search_documents(query, optimized_query_vec=None, top_n=10):
    # Inisialisasi stemmer bahasa Indonesia
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
    # Preprocessing query
    query = preprocess_text(query, stemmer)
    
    if optimized_query_vec is None:
        # Transformasi query menjadi vektor tf-idf
        query_vec = tfidf.transform([query])
    else:
        query_vec = optimized_query_vec
    
    # Menghitung cosine similarity antara query dan semua dokumen
    cosine_similarities = linear_kernel(query_vec, tfidf_matrix).flatten()
    
    # Mendapatkan indeks dokumen dengan similarity tertinggi
    related_docs_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Mendapatkan judul, teks, dan nilai similarity dari dokumen yang relevan
    results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i]) for i in related_docs_indices]
    
    # Membuat folder baru jika belum ada
    folder_baru = "hasil_pencarian"
    if not os.path.exists(folder_baru):
        os.makedirs(folder_baru)

    # Menyalin file PDF yang sesuai ke dalam folder baru
    for title, text, similarity in results:
        if ':' in title:
            filename = title.split(':')[1].strip() + ".pdf"
        else:
            filename = title.strip() + ".pdf"
        source_path = os.path.join(r"C:\Users\muham\OneDrive - Universitas Airlangga\Semester 6\Sistem Temu Kembali Informasi\Tugas dan Latihan\Final tugas akhir\File UU", filename)
        destination_path = os.path.join(folder_baru, filename)
        if os.path.exists(source_path):
            shutil.copy(source_path, destination_path)
    return results

if __name__ == "__main__":
    # Memuat vektor kueri yang dioptimalkan sebelumnya
    optimized_query_vec = pickle.load(open('optimized_query.pkl', 'rb'))
    
    query = input("Masukkan Query: ")
    print(f"Query: {query}\n")
    
    results = search_documents(query, optimized_query_vec)
    print("\n\n -- HASIL PENELUSURAN -- \n\n")
    for title, text, similarity in results:
        print(f"Judul: {title}")
        print(f"Similarity: {similarity:.4f}")
        print()

Query: pencurian



 -- HASIL PENELUSURAN -- 


Judul: Undang-Undang Nomor 2 Tahun 1984
Similarity: 0.1764

Judul: Undang-Undang Nomor 2 Tahun 1983
Similarity: 0.1745

Judul: Undang-Undang Nomor 1 Tahun 1980
Similarity: 0.1743

Judul: Undang-Undang Nomor 2 Tahun 1990
Similarity: 0.1738

Judul: Undang-Undang Nomor 3 Tahun 1984
Similarity: 0.1647

Judul: Undang-Undang Nomor 1 Tahun 1989
Similarity: 0.1566

Judul: Undang-Undang Nomor 3 Tahun 1981
Similarity: 0.1485

Judul: Undang-Undang Nomor 5 Tahun 1987
Similarity: 0.1462

Judul: Undang-Undang Nomor 18 Tahun 1992
Similarity: 0.1410

Judul: Undang-Undang Nomor 1 Tahun 1984
Similarity: 0.1374

